### Imports

In [1]:
# Pandas
import pandas as pd
# Plotly
import plotly
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)
#Mapbox
from mapboxgl.utils import *
from mapboxgl.viz import *
import os

### Analysis based on data from Facebook

In [2]:
df = pd.DataFrame(columns=list(pd.read_excel("data_sources/movement_administrative/20190711_0000.xlsx")))
for subdir, dirs, files in os.walk('data_sources/movement_administrative/'):
    for file in files:
        filepath = subdir + os.sep + file
        df = df.append(pd.read_excel(filepath))

#### Break down LineString to Latitute and Longitude Points

In [3]:
def break_linestring(linestring):
    latlng = linestring[len("LINESTRING ("):-len(")")].replace(',', '').split()
    return (latlng[0], latlng[1], latlng[2], latlng[3])

In [4]:
df['lon0'],df['lat0'],df['lon1'],df['lat1']=zip(*df.Geometry.apply(break_linestring))

In [5]:
df[["lat0", "lon0", "lat1", "lon1"]] = df[["lat0", "lon0", "lat1", "lon1"]].apply(pd.to_numeric)

In [6]:
df.head()

,Difference,Baseline: People Moving,Crisis: People Moving,Percent Change,Standard (Z) Score,Starting Region Name,Ending Region Name,Starting Location,Ending Location,Length(km),Geometry,lon0,lat0,lon1,lat1
0,17.0,48.0,65,34.693878,2.479705,Gaunaha,Gaunaha,1099795,1099795,0.167041,LINESTRING (84.57765549978836 27.2788172130663...,84.577655,27.278817,84.575965,27.278817
1,2.2,10.8,13,18.644068,0.410659,Madhuban,Pakri Dayal,1099835,1099832,1.522951,LINESTRING (85.06713867027096 26.4940214932484...,85.067139,26.494021,85.065448,26.507634
2,-29.2,321.2,292,-9.062694,-0.816899,Bankatwa,Bankatwa,1099817,1099817,0.043546,LINESTRING (85.08166170167216 26.8113056731299...,85.081662,26.811306,85.081285,26.811104
3,-140.4,790.4,650,-17.740713,-1.863768,Pakri Dayal,Pakri Dayal,1099832,1099832,0.017146,LINESTRING (85.0871506928199 26.54891580084648...,85.087151,26.548916,85.086982,26.548946
4,-201.6,1175.6,974,-17.134115,-1.422462,Chiraia,Chiraia,1099820,1099820,0.032095,LINESTRING (85.09673634263311 26.6865827344165...,85.096736,26.686583,85.096601,26.686845


### Most Difference (Negative) between Baseline and Crisis
> Captures places where people would normally move but did not move during crisis
> This might imply that either they were in dire need of attention, or felt safe where they were. More likely, it is the latter. 

In [7]:
df_neg_500 = df.sort_values("Percent Change").head(500)

In [8]:
list(df_neg_500)

['Difference',
 'Baseline: People Moving',
 'Crisis: People Moving',
 'Percent Change',
 'Standard (Z) Score',
 'Starting Region Name',
 'Ending Region Name',
 'Starting Location',
 'Ending Location',
 'Length(km)',
 'Geometry',
 'lon0',
 'lat0',
 'lon1',
 'lat1']

In [9]:
df_neg_500.head()

,Difference,Baseline: People Moving,Crisis: People Moving,Percent Change,Standard (Z) Score,Starting Region Name,Ending Region Name,Starting Location,Ending Location,Length(km),Geometry,lon0,lat0,lon1,lat1
15,-783.6,886.6,103,-88.283010,-4.0,Bairgania,Bairgania,1099843,1099843,0.533537,LINESTRING (85.28601192386054 26.7108328592309...,85.286012,26.710833,85.280679,26.711404
18,-760.5,895.5,135,-84.829894,-4.0,Bairgania,Bairgania,1099843,1099843,0.307171,LINESTRING (85.28393553703344 26.7107896699900...,85.283936,26.710790,85.287028,26.710790
23,-420.2,516.2,96,-81.245166,-4.0,Suppi,Suppi,1099844,1099844,0.022739,LINESTRING (85.34843444723013 26.6869318361575...,85.348434,26.686932,85.348663,26.686932
34,-429.4,545.4,116,-78.587116,-4.0,Suppi,Suppi,1099844,1099844,0.056457,LINESTRING (85.35638216770782 26.6850347120198...,85.356382,26.685035,85.355814,26.685035
21,-664.0,846.0,182,-78.394333,-4.0,Bairgania,Bairgania,1099843,1099843,0.059951,LINESTRING (85.27418869225455 26.7274935913037...,85.274189,26.727494,85.273585,26.727494


In [10]:
df_to_geojson(df_neg_500, filename='data_sources/temp_neg_points.geojson',
              properties=['Difference', 'Starting Region Name', 'Baseline: People Moving', 'Crisis: People Moving'],
              lat='lat0', lon='lon0', precision=3)

{'feature_count': 500,
 'filename': 'data_sources/temp_neg_points.geojson',
 'type': 'file'}

In [11]:
token  = "pk.eyJ1IjoiYXl1c2hzdWJlZGkiLCJhIjoiY2pweGp4MzV3MTl3ZzN5bzB0bzl4Y2dwMSJ9.MPh0n0FdMwYunrIHjz8lxQ"

In [12]:
viz = CircleViz('data_sources/temp_neg_points.geojson', 
                     access_token=token,
                     color_property='Crisis: People Moving',
                     color_stops=create_color_stops([0, 20, 50, 80, 100], colors='YlOrRd'),
                     color_function_type='interpolate',
                     opacity = 1,
                     radius=3,
                     center=(84, 28.5), 
                     zoom=6
                    )
viz.show()

### Most Difference (Positive) between Baseline and Crisis
> Captures places where people moved significantly.

In [13]:
df_pos_500 = df.sort_values("Percent Change", ascending=False).head(500)

In [14]:
df_pos_500.head()

,Difference,Baseline: People Moving,Crisis: People Moving,Percent Change,Standard (Z) Score,Starting Region Name,Ending Region Name,Starting Location,Ending Location,Length(km),Geometry,lon0,lat0,lon1,lat1
73,160.4,41.6,202,376.525822,4.0,Rajnagar,Madhubani,1099875,1099873,2.540838,LINESTRING (86.09082514264263 26.3629256314979...,86.090825,26.362926,86.070702,26.348890
72,153.8,45.2,199,332.900433,4.0,Rajnagar,Madhubani,1099875,1099873,2.073110,LINESTRING (86.08572034436494 26.3620443197571...,86.085720,26.362044,86.073575,26.346907
76,153.6,46.4,200,324.050633,4.0,Rajnagar,Madhubani,1099875,1099873,1.978186,LINESTRING (86.08568114690165 26.3618489641000...,86.085681,26.361849,86.070740,26.350134
84,185.6,56.4,242,323.344948,4.0,Rajnagar,Madhubani,1099875,1099873,2.425437,LINESTRING (86.08814081454787 26.3647815098907...,86.088141,26.364782,86.071434,26.348917
47,141.4,43.6,185,317.040359,4.0,Rajnagar,Madhubani,1099875,1099873,1.851601,LINESTRING (86.08275047004426 26.3627668183455...,86.082750,26.362767,86.068973,26.351592


In [15]:
df_to_geojson(df_pos_500, filename='data_sources/temp_pos_points.geojson',
              properties=['Difference', 'Starting Region Name', 'Baseline: People Moving', 'Crisis: People Moving'],
              lat='lat0', lon='lon0', precision=3)

{'feature_count': 500,
 'filename': 'data_sources/temp_pos_points.geojson',
 'type': 'file'}

In [16]:
viz = CircleViz('data_sources/temp_pos_points.geojson', 
                     access_token=token,
                     color_property='Crisis: People Moving',
                     color_stops=create_color_stops([0, 20, 50, 80, 100], colors='YlOrRd'),
                     color_function_type='interpolate',
                     opacity = 1,
                     radius=3,
                     center=(84, 28.5), 
                     zoom=6
                    )
viz.show()